In [1]:
import pandas as pd

In [4]:
# pd.read_csv('./../data/results/result_H_Q-AGLOMERADO.json')

base_str = 'H'; grouper = ['Q', 'AGLOMERADO']
info = pd.read_json('./../data/results/result_'+base_str+'_'+'-'.join(grouper)+'.json').drop('timestamp', axis = 1).drop_duplicates()
info['Q'] = pd.to_datetime(info['Q'])

In [6]:
info.nunique()

observable       9
sintetico        7
base             1
Q                7
AGLOMERADO      33
valor         2351
dtype: int64

In [8]:
info.Q.unique()

array(['2020-08-15T00:00:00.000000000', '2020-05-15T00:00:00.000000000',
       '2020-02-15T00:00:00.000000000', '2019-11-15T00:00:00.000000000',
       '2019-08-15T00:00:00.000000000', '2019-05-15T00:00:00.000000000',
       '2019-02-15T00:00:00.000000000'], dtype='datetime64[ns]')